# IMPORTS:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *

import os
from Utils import cacheOnDisk
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

done1
stop
stop
done1
stop
done
done
done
done
stop


In [2]:
import time
cdate=str(time.strftime("%d%m%Y"))
cdate


'13072018'

# generate distance matrices for phenotypes and selected TCR features

## distMat for phenotypes:

In [ ]:
#load PNP530ss9000 phenotype file:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/NewPhenotypicData/PNP530ss9000_phen_new.xlsx'
PNP530ss9000_phen_new=pd.read_excel(f1)
PNP530ss9000_phen_new=PNP530ss9000_phen_new.set_index('BD')
print len(PNP530ss9000_phen_new)

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss9000_phen_new.xlsx'
Cardio126ss9000_phen_new=pd.read_excel(f1)
Cardio126ss9000_phen_new=Cardio126ss9000_phen_new.set_index('BD')

In [ ]:
numericalphenotypes=['Age','BMI','Creatinine','WBC','eGFR_CKD-EPI_new']
categoricalphenotypes=['Gender', 'Smoking']

In [ ]:
numMetric='euclidean'
catMetric='jaccard'

### numerical phenotypes, PNP530

In [ ]:
# distMat for PNP530:
datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
distMatFolder='%s/phenDistMats' %datasetFolder
if not isdir(distMatFolder):
    makedirs(distMatFolder)
    
for n,phenotype in enumerate(numericalphenotypes):
    metric='euclidean'
    print n,phenotype
    
    df=pd.DataFrame(PNP530ss9000_phen_new[phenotype])
    df_condensed_org, distMat_square=genDistMat(df,metric)

    distMatFileSquare='%s/PNP530ss9000_%s_%s_distMat' %(distMatFolder,phenotype,metric)
    distMatFileCondensed='%s/PNP530ss9000_%s_%s_distMat_CONDENSED' %(distMatFolder,phenotype,metric)
    distMat_square.to_pickle(distMatFileSquare)
    df_condensed_org.to_pickle(distMatFileCondensed)
    

### numerical phenotypes, Cardio126

In [ ]:
# distMat for Cardio126:
datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
distMatFolder='%s/phenDistMats' %datasetFolder
if not isdir(distMatFolder):
    makedirs(distMatFolder)
    
for n,phenotype in enumerate(numericalphenotypes):
    metric='euclidean'
    print n,phenotype
    
    df=pd.DataFrame(Cardio126ss9000_phen_new[phenotype])
    df_condensed_org, distMat_square=genDistMat(df,metric)

    distMatFileSquare='%s/Cardio126ss9000_%s_%s_distMat' %(distMatFolder,phenotype,metric)
    distMatFileCondensed='%s/Cardio126ss9000_%s_%s_distMat_CONDENSED' %(distMatFolder,phenotype,metric)
    distMat_square.to_pickle(distMatFileSquare)
    df_condensed_org.to_pickle(distMatFileCondensed)
    

## distMat for TCR

In [ ]:
TCRdfNameList=['sharingMatrix_PNP530_ss9000_rep1_minNshared2_RA_onlyProductiveTrue__minSharedT2_OLtrimmed',
              'sharingMatrix_PNP530_ss9000_rep1_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed',
              'sharingMatrix_Cardio126_ss9000_rep1_minNshared2_RA_onlyProductiveTrue__minSharedT2_OLtrimmed',
              'sharingMatrix_Cardio126_ss9000_rep1_minNshared2_RA_onlyProductiveTrue__percShared10_OLtrimmed']

for n,TCRdfName in enumerate(TCRdfNameList):
    print n,TCRdfName
    if 'PNP' in TCRdfName:
        datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
        datasetName='PNP530_ss9000_rep1'
    elif 'Cardio' in TCRdfName:
        datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
        datasetName='Cardio126_ss9000_rep1'
        
    distMatFolder='%s/distanceMatrices' %datasetFolder
    if not isdir(distMatFolder):
        makedirs(distMatFolder)
    TCRdfnewName=TCRdfName.replace('sharingMatrix_','TCRdf')
    TCRdfnewName=TCRdfnewName.replace(datasetName,'')
    TCRdfnewName=TCRdfnewName.replace('_','')
    print (TCRdfnewName)


    print 'generating distance matrix for RA data:'
    file1='%s/sharingAnalysis/%s' %(datasetFolder, TCRdfName)
    TCRdf=pd.read_pickle(file1)
    metric1='braycurtis'
    print 'TCRdf sum is %s' %TCRdf.sum().sum()

    distMatFileSquare='%s/%s_%s_distMat' %(distMatFolder,TCRdfnewName,metric1)
    distMatFileCondensed='%s/%s_%s_distMat_CONDENSED' %(distMatFolder,TCRdfnewName,metric1)

    df_condensed_org, distMat_square=genDistMat(TCRdf,metric1)
    print 'distMat_square sum is %s' %distMat_square.sum().sum()
    distMat_square.to_pickle(distMatFileSquare)
    df_condensed_org.to_pickle(distMatFileCondensed)  


    print 'generating distance matrix for binary data:'
    TCRdfName_binary=TCRdfName+'_binary'
    TCRdfnewName_binary=TCRdfnewName+'_binary'

    file2='%s/sharingAnalysis/%s' %(datasetFolder, TCRdfName_binary)
    TCRdf_binary=pd.read_pickle(file2)
    metric2='jaccard'

    distMatFileSquare_binary='%s/%s_%s_distMat' %(distMatFolder,TCRdfnewName_binary,metric2)
    distMatFileCondensed_binary='%s/%s_%s_distMat_CONDENSED' %(distMatFolder,TCRdfnewName_binary,metric2)

    df_condensed_org_binary, distMat_square_binary=genDistMat(TCRdf_binary,metric2)
    distMat_square_binary.to_pickle(distMatFileSquare_binary)
    df_condensed_org_binary.to_pickle(distMatFileCondensed_binary) 
    
    print 'done'



# Permanova for gender and smoking effect on TCR features

## Generate dummy variables:

### PNP samples:

In [ ]:
PNP530ss9000_phen_new.Smoking.value_counts(dropna=False)

In [ ]:
toDummyColList=['Gender','Smoking','PCRplate']
df=PNP530ss9000_phen_new.copy()
PNP530ss9000_phen_new_dummies=gen_dummies(df,toDummyColList)
PNP530ss9000_phen_new_dummies.head()

In [ ]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/NewPhenotypicData/PNP530ss9000_phen_new_dummies.xlsx'
PNP530ss9000_phen_new_dummies.to_excel(f1)

### cardio samples:

In [3]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss9000_phen_new.xlsx'
Cardio126ss9000_phen_new=pd.read_excel(f1)
Cardio126ss9000_phen_new=Cardio126ss9000_phen_new.set_index('BD')

In [4]:
Cardio126ss9000_phen_new.Smoking.value_counts(dropna=False)

No      45
Yes     36
Past    24
NaN      3
Name: Smoking, dtype: int64

In [7]:
Cardio126ss9000_phen_new_dummies=Cardio126ss9000_phen_new.copy()

In [8]:
PCIdict={0:'0',1:'1',2:'2+',3:'2+',4:'2+',np.nan:np.nan}
Cardio126ss9000_phen_new_dummies['PreviousPCImapped']=Cardio126ss9000_phen_new_dummies['Previous PCI'].map(PCIdict)

In [9]:
LVEFdict={0:1,1:1,2:1,3:2,4:2,5:2,6:1,np.nan:np.nan}
Cardio126ss9000_phen_new_dummies['LVEFmapped']=Cardio126ss9000_phen_new_dummies['LVEF'].map(LVEFdict) 

In [10]:
Cardio126ss9000_phen_new_dummies['PreviousPCImapped'].value_counts(dropna=False)

0      60
NaN    25
1      14
2+      9
Name: PreviousPCImapped, dtype: int64

In [11]:
toDummyColList=['Gender','Smoking','PCRplate','DM','Diagnosis','Known CAD','Previous CABG','PreviousPCImapped','LVEFmapped']
df=Cardio126ss9000_phen_new_dummies.copy()
Cardio126ss9000_phen_new_dummies=gen_dummies(df,toDummyColList)
Cardio126ss9000_phen_new_dummies.head()

original number of columns is 23
number of toDummy columns is 9
number of columns after dummy generation is 36


,RegistrationCode,Age,BMI,Creatinine,DM,Diagnosis,Gender,Smoking,PCRplate,isCardio,...,PCRplate_Plate8,PCRplate_Plate9,DM_Yes,Diagnosis_STEMI,Diagnosis_UA,Known CAD_1.0,Previous CABG_1.0,PreviousPCImapped_1,PreviousPCImapped_2+,LVEFmapped_2.0
BD,,,,,,,,,,,,,,,,,,,,,
BD1003,3278763.0,67.0,28.634039,1.15,Yes,NSTEMI,Male,Past,Plate9,1,...,0,1,1,0,0,0,0,0,0,0
BD993,3825462.0,66.0,32.812500,0.85,Yes,NSTEMI,Female,Past,Plate9,1,...,0,1,1,0,0,1,0,0,1,0
BD1054,3493727.0,67.0,27.774271,1.09,No,NSTEMI,Male,No,Plate9,1,...,0,1,0,0,0,0,0,0,0,0
BD983,3040383.0,57.0,27.170771,1.02,No,STEMI,Male,No,Plate8,1,...,1,0,0,1,0,0,0,0,0,0
BD1013,3457111.0,75.0,31.487889,1.31,No,STEMI,Male,Past,Plate9,1,...,0,1,0,1,0,1,0,0,0,0


In [15]:
for col in Cardio126ss9000_phen_new_dummies.columns:
    Cardio126ss9000_phen_new_dummies=Cardio126ss9000_phen_new_dummies.rename(columns={col:col.replace('.0','')})
Cardio126ss9000_phen_new_dummies.head()

,RegistrationCode,Age,BMI,Creatinine,DM,Diagnosis,Gender,Smoking,PCRplate,isCardio,...,PCRplate_Plate8,PCRplate_Plate9,DM_Yes,Diagnosis_STEMI,Diagnosis_UA,Known CAD_1,Previous CABG_1,PreviousPCImapped_1,PreviousPCImapped_2+,LVEFmapped_2
BD,,,,,,,,,,,,,,,,,,,,,
BD1003,3278763.0,67.0,28.634039,1.15,Yes,NSTEMI,Male,Past,Plate9,1,...,0,1,1,0,0,0,0,0,0,0
BD993,3825462.0,66.0,32.812500,0.85,Yes,NSTEMI,Female,Past,Plate9,1,...,0,1,1,0,0,1,0,0,1,0
BD1054,3493727.0,67.0,27.774271,1.09,No,NSTEMI,Male,No,Plate9,1,...,0,1,0,0,0,0,0,0,0,0
BD983,3040383.0,57.0,27.170771,1.02,No,STEMI,Male,No,Plate8,1,...,1,0,0,1,0,0,0,0,0,0
BD1013,3457111.0,75.0,31.487889,1.31,No,STEMI,Male,Past,Plate9,1,...,0,1,0,1,0,1,0,0,0,0


In [16]:
f1='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss9000_phen_new_dummies.xlsx'
Cardio126ss9000_phen_new_dummies.to_excel(f1)

## running permanova:

In [ ]:
# Gender and smoking for PNP and Cardio:
nPerm=9999
removeSameUser=True


# DEFINE PHENOTYPES AND DATASETS:
phenotypeList=['Gender_Male','Smoking_Past','Smoking_Yes']
datasetList=['PNP530','Cardio126']

for phenotype in phenotypeList:

    for dataset in datasetList:
        
        if 'PNP' in dataset: #DEFINE RELEVANT DATASET FOLDER, NAME AND PHENOTYPE DF FILE
            print 'dataset is PNP'
            datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
            datasetName='PNP530_ss9000_rep1'
            phenotypeDFfile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/NewPhenotypicData/PNP530ss9000_phen_new_dummies.xlsx'
            # GENERATE FOLDER FOR RESULTS:
            permanovaDFfolder='%s/TCR_phenotype_relations/permanovaDFs' %datasetFolder
            if not isdir(permanovaDFfolder):
                makedirs(permanovaDFfolder)
        elif 'Cardio' in dataset:
            print 'dataset is Cardio'
            datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
            datasetName='Cardio126_ss9000_rep1'
            phenotypeDFfile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss9000_phen_new_dummies.xlsx'
            
            permanovaDFfolder='%s/TCR_phenotype_relations/permanovaDFs' %datasetFolder
            if not isdir(permanovaDFfolder):
                makedirs(permanovaDFfolder)
                
                
        phenotypeDF=pd.read_excel(phenotypeDFfile)
        phenotypeDF=phenotypeDF.set_index('BD')
        
        #EXTRACT ALL TCR distMat to check:
        TCRdistMatFolder='%s/distanceMatrices' %datasetFolder
        TCRdistMatNameList=[f for f in listdir(TCRdistMatFolder) if isfile(join(TCRdistMatFolder,f))]
        TCRdistMatNameList=[f for f in TCRdistMatNameList if '_CONDENSED' not in f]
        
        print phenotype, dataset
        print 'number of distMats to analyze in folder is %s' %len(TCRdistMatNameList)
        print TCRdistMatNameList
        
        for TCRdistMatName in TCRdistMatNameList:
            print TCRdistMatName
            TCRdistMat=pd.read_pickle(join(TCRdistMatFolder,TCRdistMatName))
            
            print 'TCRdistMat head:'
            print TCRdistMat.iloc[:5,:5]
            
            newName=TCRdistMatName.replace('minNshared2RAonlyProductiveTrue','')
            
            phenotypeDF=phenotypeDF[phenotypeDF[phenotype].notnull()]
            print 'phenotype head:'
            print phenotypeDF[phenotype].head()
            
            print 'conducting permanova'
            
            permAnovaResDF=permanova_withWrapper(TCRdistMat,newName,phenotypeDF,phenotype, 
                                         nPerm,removeSameUser,permanovaDFfolder)
            
            print 'done!'
            

            
        
        
        

In [13]:
Cardio126ss9000_phen_new_dummies.columns.values

array([u'RegistrationCode', u'Age', u'BMI', u'Creatinine', u'DM',
       u'Diagnosis', u'Gender', u'Smoking', u'PCRplate', u'isCardio',
       u'nTemplates', u'eGFR by CKD-EPI', u'WBC', u'CRP', u'Known CAD',
       u'Previous PCI', u'Previous CABG', u'LVEF', u'Initial Troponin',
       u'Maximal Troponin', u'eGFR_CKD-EPI_new', 'PreviousPCImapped',
       'LVEFmapped', u'Gender_Male', u'Smoking_Past', u'Smoking_Yes',
       u'PCRplate_Plate8', u'PCRplate_Plate9', u'DM_Yes',
       u'Diagnosis_STEMI', u'Diagnosis_UA', 'Known CAD_1.0',
       'Previous CABG_1.0', 'PreviousPCImapped_1', 'PreviousPCImapped_2+',
       'LVEFmapped_2.0'], dtype=object)

In [18]:
# cardio-specific parameters:
nPerm=99
removeSameUser=True


# DEFINE PHENOTYPES AND DATASETS:
phenotypeList=['PreviousPCImapped','LVEFmapped', 'DM_Yes', 'Diagnosis_STEMI', 'Diagnosis_UA', 'Known CAD_1',
       'Previous CABG_1', 'PreviousPCImapped_1', 'PreviousPCImapped_2+','LVEFmapped_2']
datasetList=['Cardio126']

for phenotype in phenotypeList:

    for dataset in datasetList:
        
        if 'PNP' in dataset: #DEFINE RELEVANT DATASET FOLDER, NAME AND PHENOTYPE DF FILE
            print 'dataset is PNP'
            datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
            datasetName='PNP530_ss9000_rep1'
            phenotypeDFfile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/NewPhenotypicData/PNP530ss9000_phen_new_dummies.xlsx'
            # GENERATE FOLDER FOR RESULTS:
            permanovaDFfolder='%s/TCR_phenotype_relations/permanovaDFs' %datasetFolder
            if not isdir(permanovaDFfolder):
                makedirs(permanovaDFfolder)
        elif 'Cardio' in dataset:
            print 'dataset is Cardio'
            datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
            datasetName='Cardio126_ss9000_rep1'
            phenotypeDFfile='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/Cardio126ss9000_phen_new_dummies.xlsx'
            
            permanovaDFfolder='%s/TCR_phenotype_relations/CardioPhenotypes/permanovaDFs' %datasetFolder
            if not isdir(permanovaDFfolder):
                makedirs(permanovaDFfolder)
                
                
        phenotypeDF=pd.read_excel(phenotypeDFfile)
        phenotypeDF=phenotypeDF.set_index('BD')
        
        #EXTRACT ALL TCR distMat to check:
        TCRdistMatFolder='%s/distanceMatrices' %datasetFolder
        TCRdistMatNameList=[f for f in listdir(TCRdistMatFolder) if isfile(join(TCRdistMatFolder,f))]
        TCRdistMatNameList=[f for f in TCRdistMatNameList if '_CONDENSED' not in f]
        
        print phenotype, dataset
        print 'number of distMats to analyze in folder is %s' %len(TCRdistMatNameList)
        print TCRdistMatNameList
        
        for TCRdistMatName in TCRdistMatNameList:
            print TCRdistMatName
            TCRdistMat=pd.read_pickle(join(TCRdistMatFolder,TCRdistMatName))
            
            print 'TCRdistMat head:'
            print TCRdistMat.iloc[:5,:5]
            
            newName=TCRdistMatName.replace('minNshared2RAonlyProductiveTrue','')
            
            phenotypeDF=phenotypeDF[phenotypeDF[phenotype].notnull()]
            print 'phenotype head:'
            print phenotypeDF[phenotype].head()
            
            print 'conducting permanova'
            
            permAnovaResDF=permanova_withWrapper(TCRdistMat,newName,phenotypeDF,phenotype, 
                                         nPerm,removeSameUser,permanovaDFfolder)
            
            print 'done!'
            

            
        
        
        

dataset is Cardio
PreviousPCImapped Cardio126
number of distMats to analyze in folder is 4
['TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_braycurtis_distMat', 'TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_binary_jaccard_distMat', 'TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_binary_jaccard_distMat', 'TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_braycurtis_distMat']
TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_braycurtis_distMat
TCRdistMat head:
Sample    BD1000    BD1001    BD1003    BD1004    BD1005
Sample                                                  
BD1000  0.000000  0.877075  0.928868  0.921228  0.934772
BD1001  0.877075  0.000000  0.807229  0.866344  0.911733
BD1003  0.928868  0.807229  0.000000  0.820096  0.957281
BD1004  0.921228  0.866344  0.820096  0.000000  0.812764
BD1005  0.934772  0.911733  0.957281  0.812764  0.000000
phenotype head:
BD
BD1003     0
BD993     2+
BD1054     0
BD983      0
BD1013     0
Name:

Diagnosis_STEMI Cardio126
number of distMats to analyze in folder is 4
['TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_braycurtis_distMat', 'TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_binary_jaccard_distMat', 'TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_binary_jaccard_distMat', 'TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_braycurtis_distMat']
TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_braycurtis_distMat
TCRdistMat head:
Sample    BD1000    BD1001    BD1003    BD1004    BD1005
Sample                                                  
BD1000  0.000000  0.877075  0.928868  0.921228  0.934772
BD1001  0.877075  0.000000  0.807229  0.866344  0.911733
BD1003  0.928868  0.807229  0.000000  0.820096  0.957281
BD1004  0.921228  0.866344  0.820096  0.000000  0.812764
BD1005  0.934772  0.911733  0.957281  0.812764  0.000000
phenotype head:
BD
BD1003    0
BD993     0
BD1054    0
BD983     1
BD1013    1
Name: Diagnosis_STEMI, dtype: 

phenotype head:
BD
BD1003    0
BD993     1
BD1054    0
BD983     0
BD1013    1
Name: Known CAD_1, dtype: int64
conducting permanova
load and process distMat and phenotype...
conducting permanova...
0.46
0.46
done!
TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_braycurtis_distMat
TCRdistMat head:
Sample    BD1000    BD1001    BD1003    BD1004    BD1005
Sample                                                  
BD1000  0.000000  0.987776  0.985229  0.976875  0.979416
BD1001  0.987776  0.000000  0.981562  0.981879  0.982358
BD1003  0.985229  0.981562  0.000000  0.964961  0.977822
BD1004  0.976875  0.981879  0.964961  0.000000  0.977434
BD1005  0.979416  0.982358  0.977822  0.977434  0.000000
phenotype head:
BD
BD1003    0
BD993     1
BD1054    0
BD983     0
BD1013    1
Name: Known CAD_1, dtype: int64
conducting permanova
load and process distMat and phenotype...
conducting permanova...
0.48
0.48
done!
dataset is Cardio
Previous CABG_1 Cardio126
number of distMats to analyze in fol

BD
BD1003    0
BD993     1
BD1054    0
BD983     0
BD1013    0
Name: PreviousPCImapped_2+, dtype: int64
conducting permanova
This permanova test result already exist in folder
done!
dataset is Cardio
LVEFmapped_2 Cardio126
number of distMats to analyze in folder is 4
['TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_braycurtis_distMat', 'TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_binary_jaccard_distMat', 'TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_binary_jaccard_distMat', 'TCRdfminNshared2RAonlyProductiveTrueminSharedT2OLtrimmed_braycurtis_distMat']
TCRdfminNshared2RAonlyProductiveTruepercShared10OLtrimmed_braycurtis_distMat
TCRdistMat head:
Sample    BD1000    BD1001    BD1003    BD1004    BD1005
Sample                                                  
BD1000  0.000000  0.877075  0.928868  0.921228  0.934772
BD1001  0.877075  0.000000  0.807229  0.866344  0.911733
BD1003  0.928868  0.807229  0.000000  0.820096  0.957281
BD1004  0.921228  0.866344

### get results for PNP530:

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
datasetName='PNP530_ss9000_rep1'
permanovaDFfolder='%s/TCR_phenotype_relations/permanovaDFs' %datasetFolder


dfs_folder=permanovaDFfolder
PermanovaResults1=concat_summarizing_dfs(dfs_folder)

nTests=len(PermanovaResults1)
PermanovaResults1=PermanovaResults1.sort_values(by='p')
PermanovaResults1=add_corrected_pValues(PermanovaResults1,pValueColumn='p',nTests=nTests,FDR=0.1)
PermanovaResults1['cohort']='PNP'
PermanovaResults1

### get results for Cardio126:

In [19]:
# common phenotypes:

datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
datasetName='Cardio126_ss9000_rep1'
permanovaDFfolder='%s/TCR_phenotype_relations/permanovaDFs' %datasetFolder


dfs_folder=permanovaDFfolder
PermanovaResults2=concat_summarizing_dfs(dfs_folder)

nTests=len(PermanovaResults2)
PermanovaResults2=PermanovaResults2.sort_values(by='p')
PermanovaResults2=add_corrected_pValues(PermanovaResults2,pValueColumn='p',nTests=nTests,FDR=0.1)
PermanovaResults2['cohort']='Cardio'
PermanovaResults2

number of dfs in directory: 12
the length of df list is 12


,featureName,phenotype,nPerm,removeSameUser,s,p,Sig by bonferroni corrected pVal,sig. by FDR=0.1,cohort
0,TCRdfpercShared10OLtrimmed_braycurtis_distMat,Smoking_Past,9999.0,True,1.235788,0.0534,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_binary_jaccard_distMat,Smoking_Past,9999.0,True,1.149269,0.0651,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_braycurtis_distMat,Gender_Male,9999.0,True,1.012259,0.2186,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_braycurtis_distMat,Gender_Male,9999.0,True,1.095159,0.2379,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_binary_jaccard_distMat,Smoking_Yes,9999.0,True,1.056360,0.2734,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_binary_jaccard_distMat,Gender_Male,9999.0,True,1.048124,0.2999,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_braycurtis_distMat,Smoking_Yes,9999.0,True,1.040163,0.3862,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_binary_jaccard_distMat,Gender_Male,9999.0,True,1.001775,0.3953,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_braycurtis_distMat,Smoking_Past,9999.0,True,0.999139,0.4805,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_binary_jaccard_distMat,Smoking_Past,9999.0,True,0.991405,0.7573,0,0,Cardio


In [20]:
# cardio phenotypes:

datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
datasetName='Cardio126_ss9000_rep1'
permanovaDFfolder='%s/TCR_phenotype_relations/CardioPhenotypes/permanovaDFs' %datasetFolder


dfs_folder=permanovaDFfolder
PermanovaResults2=concat_summarizing_dfs(dfs_folder)

nTests=len(PermanovaResults2)
PermanovaResults2=PermanovaResults2.sort_values(by='p')
PermanovaResults2=add_corrected_pValues(PermanovaResults2,pValueColumn='p',nTests=nTests,FDR=0.1)
PermanovaResults2['cohort']='Cardio'
PermanovaResults2

number of dfs in directory: 40
the length of df list is 40


,featureName,phenotype,nPerm,removeSameUser,s,p,Sig by bonferroni corrected pVal,sig. by FDR=0.1,cohort
0,TCRdfpercShared10OLtrimmed_binary_jaccard_distMat,PreviousPCImapped,99.0,True,1.108020,0.05,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_braycurtis_distMat,PreviousPCImapped,99.0,True,1.151429,0.08,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_braycurtis_distMat,DM_Yes,99.0,True,1.032927,0.08,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_binary_jaccard_distMat,PreviousPCImapped_2+,99.0,True,1.122289,0.09,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_binary_jaccard_distMat,DM_Yes,99.0,True,1.016578,0.10,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_braycurtis_distMat,PreviousPCImapped_2+,99.0,True,1.121359,0.17,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_braycurtis_distMat,PreviousPCImapped_1,99.0,True,1.135147,0.18,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_binary_jaccard_distMat,Previous CABG_1,99.0,True,1.005163,0.23,0,0,Cardio
0,TCRdfpercShared10OLtrimmed_binary_jaccard_distMat,PreviousPCImapped_1,99.0,True,1.083507,0.24,0,0,Cardio
0,TCRdfminSharedT2OLtrimmed_binary_jaccard_distMat,Known CAD_1,99.0,True,1.005516,0.27,0,0,Cardio


### combine results from cardio and PNP530, only 9999 perms

In [ ]:
AllPermanova=pd.concat([PermanovaResults1,PermanovaResults2])
AllPermanova=AllPermanova[AllPermanova['nPerm']==9999]


nTests=len(AllPermanova)
AllPermanova=AllPermanova.sort_values(by='p')
AllPermanova=add_corrected_pValues(AllPermanova,pValueColumn='p',nTests=nTests,FDR=0.1)

f1='%s/TCR_phenotype_relations/AllPermanova.xlsx' %datasetFolder
AllPermanova.to_excel(f1)


AllPermanova

# mantel test for age, BMI, createnine, WBC and eGFR effect on TCR features

## define function:

the function mantel_withWrapper was copied to feature_phenotype_functions.py

## run mantel test:

In [ ]:
phenotypeList=['Age','BMI','Creatinine','WBC','eGFR_CKD-EPI_new']
datasetList=['PNP530','Cardio126']


nPerm=9999
method='spearman'
alternative='greater'
removeSameUser=True



for dataset in datasetList:

    if 'PNP' in dataset: #DEFINE RELEVANT DATASET FOLDER, NAME AND PHENOTYPE DF FILE
        print 'dataset is PNP'
        datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
        datasetName='PNP530_ss9000_rep1'
          # GENERATE FOLDER FOR RESULTS:
        mantelDFfolder='%s/TCR_phenotype_relations/mantelDFs' %datasetFolder
        if not isdir(mantelDFfolder):
            makedirs(mantelDFfolder)
    elif 'Cardio' in dataset:
        print 'dataset is Cardio'
        datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
        datasetName='Cardio126_ss9000_rep1'
          # GENERATE FOLDER FOR RESULTS:
        mantelDFfolder='%s/TCR_phenotype_relations/mantelDFs' %datasetFolder
        if not isdir(mantelDFfolder):
            makedirs(mantelDFfolder)
            
            
    for phenotype in phenotypeList:
        phenotypedistMatFolder='%s/phenDistMats' %datasetFolder
        phenotypedistMatName='%sss9000_%s_euclidean_distMat' %(dataset,phenotype)
        phenotype_distMat=pd.read_pickle('%s/%s' %(phenotypedistMatFolder,phenotypedistMatName))
            
            
        #EXTRACT ALL TCR distMat to check:
        TCRdistMatFolder='%s/distanceMatrices' %datasetFolder
        TCRdistMatNameList=[f for f in listdir(TCRdistMatFolder) if isfile(join(TCRdistMatFolder,f))]
        TCRdistMatNameList=[f for f in TCRdistMatNameList if '_CONDENSED' not in f]

        print 'number of TCR distMats to analyze in folder is %s' %len(TCRdistMatNameList)
#             print TCRdistMatNameList

        for TCRdistMatName in TCRdistMatNameList:
        
            print phenotype,dataset, 
            print phenotypedistMatName, TCRdistMatName
#                 print TCRdistMatName
            TCR_distMat=pd.read_pickle(join(TCRdistMatFolder,TCRdistMatName))
            newName=TCRdistMatName.replace('minNshared2RAonlyProductiveTrue','')

            feature_distMat=TCR_distMat
            feature_name=newName
#                 print feature_name
            phenotype_name=phenotypedistMatName
#                 print phenotype_name


            mantel_withWrapper(feature_distMat,feature_name,phenotype_distMat,phenotype_name, mantelDFfolder, 
                                   nPerm,method,alternative, removeSameUser, i=None,j=None)

In [ ]:
phenotypeList=['Age','BMI','Creatinine']
datasetList=['PNP530','Cardio126']


nPerm=99
method='spearman'
alternative='greater'
removeSameUser=True



for dataset in datasetList:

    if 'PNP' in dataset: #DEFINE RELEVANT DATASET FOLDER, NAME AND PHENOTYPE DF FILE
        print 'dataset is PNP'
        datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
        datasetName='PNP530_ss9000_rep1'
          # GENERATE FOLDER FOR RESULTS:
        mantelDFfolder='%s/TCR_phenotype_relations/mantelDFs' %datasetFolder
        if not isdir(mantelDFfolder):
            makedirs(mantelDFfolder)
    elif 'Cardio' in dataset:
        print 'dataset is Cardio'
        datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
        datasetName='Cardio126_ss9000_rep1'
          # GENERATE FOLDER FOR RESULTS:
        mantelDFfolder='%s/TCR_phenotype_relations/mantelDFs' %datasetFolder
        if not isdir(mantelDFfolder):
            makedirs(mantelDFfolder)
            
            
    for phenotype in phenotypeList:
        phenotypedistMatFolder='%s/phenDistMats' %datasetFolder
        phenotypedistMatName='%sss9000_%s_euclidean_distMat' %(dataset,phenotype)
        phenotype_distMat=pd.read_pickle('%s/%s' %(phenotypedistMatFolder,phenotypedistMatName))
            
            
        #EXTRACT ALL TCR distMat to check:
        TCRdistMatFolder='%s/distanceMatrices' %datasetFolder
        TCRdistMatNameList=[f for f in listdir(TCRdistMatFolder) if isfile(join(TCRdistMatFolder,f))]
        TCRdistMatNameList=[f for f in TCRdistMatNameList if '_CONDENSED' not in f]

        print 'number of TCR distMats to analyze in folder is %s' %len(TCRdistMatNameList)
#             print TCRdistMatNameList

        for TCRdistMatName in TCRdistMatNameList:
        
            print phenotype,dataset, 
            print phenotypedistMatName, TCRdistMatName
#                 print TCRdistMatName
            TCR_distMat=pd.read_pickle(join(TCRdistMatFolder,TCRdistMatName))
            newName=TCRdistMatName.replace('minNshared2RAonlyProductiveTrue','')

            feature_distMat=TCR_distMat
            feature_name=newName
#                 print feature_name
            phenotype_name=phenotypedistMatName
#                 print phenotype_name


            mantel_withWrapper(feature_distMat,feature_name,phenotype_distMat,phenotype_name, mantelDFfolder, 
                                   nPerm,method,alternative, removeSameUser, i=None,j=None)

### get results for PNP530:

In [ ]:
datasetFolder='%s/TCR_real_data/PNP530_SubSampled9000data_rep1' %MyPath
datasetName='PNP530_ss9000_rep1'
mantelDFfolder='%s/TCR_phenotype_relations/mantelDFs' %datasetFolder


dfs_folder=mantelDFfolder
mantelResults1=concat_summarizing_dfs(dfs_folder)

nTests=len(mantelResults1)
mantelResults1=mantelResults1.sort_values(by='p')
mantelResults1=add_corrected_pValues(mantelResults1,pValueColumn='p',nTests=nTests,FDR=0.1)

f1='%s/TCR_phenotype_relations/mantelResults1.xlsx' %datasetFolder
mantelResults1.to_excel(f1)

mantelResults1

### get results for Cardio126:

In [ ]:
datasetFolder='%s/TCR_real_data/CardioSamples/Cardio126_SubSampled9000data_rep1' %MyPath
datasetName='Cardio126_ss9000_rep1'
mantelDFfolder='%s/TCR_phenotype_relations/mantelDFs' %datasetFolder


dfs_folder=mantelDFfolder
mantelResults2=concat_summarizing_dfs(dfs_folder)

nTests=len(mantelResults2)
mantelResults2=mantelResults2.sort_values(by='p')
mantelResults2=add_corrected_pValues(mantelResults2,pValueColumn='p',nTests=nTests,FDR=0.1)

f1='%s/TCR_phenotype_relations/mantelResults2.xlsx' %datasetFolder
mantelResults2.to_excel(f1)

mantelResults2

### combine results from cardio and PNP530, only 9999 perms

In [ ]:
AllMantel=pd.concat([mantelResults1,mantelResults2])
AllMantel=AllMantel[AllMantel['nPerm']==9999]


nTests=len(AllMantel)
AllMantel=AllMantel.sort_values(by='p')
AllMantel=add_corrected_pValues(AllMantel,pValueColumn='p',nTests=nTests,FDR=0.1)

f1='%s/TCR_phenotype_relations/AllMantel.xlsx' %datasetFolder
AllMantel.to_excel(f1)


AllMantel